In [ ]:
# ЗАГРУЗКА АРХИВА, СОДЕРЖАЩЕГО БИБЛИОТЕКУ ИЗОБРАЖЕНИЙ, АРХИВ RAR
!apt-get install unrar
import os
import shutil
from google.colab import files

# Путь к папке
OUTPUT_FOLDER = "/content/найденные"

# Очистка папки
if os.path.exists(OUTPUT_FOLDER):
    shutil.rmtree(OUTPUT_FOLDER)  # Удалить папку со всем содержимым
os.mkdir(OUTPUT_FOLDER)  # Создать пустую папку

# Функция очистки папки
def clear_folder(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)  # Удалить папку со всем содержимым
    os.mkdir(folder_path)  # Создать пустую папку

# Путь к папке, которая будет очищаться
FOLDER_PATH = "/content/folder"
clear_folder(FOLDER_PATH)  # Очистка папки

# Путь к папке, которая будет очищаться
OUTPUT_FOLDER = "/content/найденные"
clear_folder(OUTPUT_FOLDER)  # Очистка папки

# Загрузка архива .rar
uploaded = files.upload()  # Выберите архив .rar

# Распаковка архива в папку
rar_file = list(uploaded.keys())[0]  # Имя загруженного архива
!unrar x -o+ "{rar_file}" {FOLDER_PATH}/



In [ ]:
# ЗАПУСК ПОИСКА ИЗОБРАЖЕНИЯ
import cv2
import os
import shutil
from google.colab import files  # Для загрузки файлов пользователем

# Путь к папке с изображениями
FOLDER_PATH = "/content/folder"  # Загрузите изображения сюда
# Папка для сохранения найденных изображений
OUTPUT_FOLDER = "/content/найденные"

def load_images_from_folder(folder_path):
    images = []
    filenames = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
            filenames.append(filename)
    return images, filenames

def find_most_similar_images(target_image_path, images, filenames, top_n=10):
    # Загрузка и обработка целевого изображения
    target_image = cv2.imread(target_image_path, cv2.IMREAD_GRAYSCALE)

    # Инициализация ORB детектора
    orb = cv2.ORB_create()

    # Извлечение признаков и дескрипторов целевого изображения
    keypoints1, descriptors1 = orb.detectAndCompute(target_image, None)

    matches_data = []

    # Инициализация BFMatcher для поиска ближайших соседей
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    for i, img in enumerate(images):
        # Извлечение признаков и дескрипторов для изображения из папки
        keypoints2, descriptors2 = orb.detectAndCompute(img, None)

        # Сравнение дескрипторов с использованием BFMatcher
        matches = bf.match(descriptors1, descriptors2)

        # Сортировка совпадений по расстоянию
        matches = sorted(matches, key=lambda x: x.distance)

        # Определение количества совпадений
        num_matches = len(matches)

        # Сохранение информации о совпадении
        matches_data.append((filenames[i], num_matches))

    # Сортировка изображений по количеству совпадений в порядке убывания
    matches_data = sorted(matches_data, key=lambda x: x[1], reverse=True)

    # Возвращаем топ N совпадений
    return matches_data[:top_n]

# Очистка папки перед записью
def clear_output_folder():
    if os.path.exists(OUTPUT_FOLDER):
        shutil.rmtree(OUTPUT_FOLDER)  # Удалить папку со старым содержимым
    os.mkdir(OUTPUT_FOLDER)  # Создать пустую папку

def main():
    # Очистка папки "найденные"
    clear_output_folder()
    # Загрузка целевого изображения
    print("Загрузите исходное изображение:")
    uploaded = files.upload()  # Пользователь загружает файл
    target_image_path = list(uploaded.keys())[0]

    # Загрузка изображений из папки
    images, filenames = load_images_from_folder(FOLDER_PATH)

    # Очистка папки "найденные" перед записью
    if os.path.exists(OUTPUT_FOLDER):
      shutil.rmtree(OUTPUT_FOLDER)  # Удалить старые файлы
    os.mkdir(OUTPUT_FOLDER)  # Создать пустую папку

    # Поиск самых похожих изображений
    top_matches = find_most_similar_images(target_image_path, images, filenames, top_n=10)

    # Сохранение и вывод наиболее похожих изображений
    for filename, num_matches in top_matches:
        src_path = os.path.join(FOLDER_PATH, filename)
        dest_path = os.path.join(OUTPUT_FOLDER, filename)
        shutil.copy(src_path, dest_path)  # Копирование файла в папку "найденные"
        print(f"Скопировано: {filename} с {num_matches} совпадениями.")

    print(f"Папка с найденными изображениями: {OUTPUT_FOLDER}")

if __name__ == "__main__":
    main()

#ВЫГРУЗКА АРХИВА С ОТОБРАННЫМИ ИЗОБРАЖЕНИЯМИ (2 шт)
!zip -r найденные.zip /content/найденные
files.download("найденные.zip")


In [ ]:
#ОЧИСТКА СИСТЕМНОЙ ПАПКИ, КУДА БЫЛИ ОТОБРАНЫ ИЗОБРАЖЕНИЯ
def clear_найденные():
  !rm -rf найденные.zip /content/найденные
#очистка папки, куда сохранялись найденные файлы
clear_найденные()
#if clear_найденные is True:
print("папка /найденные очищена, загрузите очередной файл с изображением")


папка /найденные очищена, загрузите очередной файл с изображением
